## Environment Set-up 😀

You have to run it with Nvidia GPU, though...

In [ ]:
# # !conda activate aidraw
# !pip install gradio

In [ ]:
# !pip install --upgrade --force-reinstall diffusers==0.14.0

In [ ]:
# !pip install --upgrade --force-reinstall transformers

In [ ]:
# !pip install opencv-contrib-python
# !pip install controlnet_aux
# !pip install requests==2.28.1 

In [ ]:
# !pip install accelerate

In [ ]:
# !pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --user --extra-index-url https://download.pytorch.org/whl/cu113

In [2]:
import torch
torch.cuda.is_available()

False

## 0-to-Sketch LoRA (Deprycated)

In [ ]:
# import requests 
# from PIL import Image 
# import gradio as gr 
# from io import BytesIO 
# url = "https://stablediffusionapi.com/api/v3/text2img" 
# title = """&lt;h2&gt;&lt;center&gt;Text to Image Generation with Stable Diffusion API&lt;/center&gt;&lt;/h2&gt;""" 
# description = """#### Get the API key by signing up here [Stable Diffusion API](https://stablediffusionapi.com).""" 
# def get_image(key, prompt, inference_steps, filter): 
#     payload = { "key": key, "prompt": prompt, "negative_prompt": "((out of frame)), ((extra fingers)), mutated hands, ((poorly drawn hands)), ((poorly drawn face)), (((mutation))), (((deformed))), (((tiling))), ((naked)), ((tile)), ((fleshpile)), ((ugly)), (((abstract))), blurry, ((bad anatomy)), ((bad proportions)), ((extra limbs)), cloned face, (((skinny))), glitchy, ((extra breasts)), ((double torso)), ((extra arms)), ((extra hands)), ((mangled fingers)), ((missing breasts)), (missing lips), ((ugly face)), ((fat)), ((extra legs)), anime", "width": "512", "height": "512", "samples": "1", "num_inference_steps": inference_steps,"safety_checker": filter,"enhance_prompt": "yes","guidance_scale": 7.5} 
#     headers = {} 
#     response = requests.request("POST", url, headers=headers, data=payload) 
#     url1 = str(json.loads(response.text)['output'][0]) 
#     r = requests.get(url1) 
#     i = Image.open(BytesIO(r.content)) 
#     return i 
# demo = gr.Interface(fn=get_image, inputs = [gr.Textbox(label="Enter API key"), 
#                                             gr.Textbox(label="Enter the Prompt"), gr.Number(label="Enter number of steps"),gr.Checkbox(label="Safety filter")], outputs = gr.Image(type='pil'), title = title, description = description)
# demo.launch(debug='True')


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


## ControlNet Pipeline 🚀

In [1]:
import cv2
from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
import numpy as np
from diffusers.utils import load_image

In [2]:
import cv2
import numpy as np



# Reshape the Image to fit ControlNet model input (512*512)
def re_image(image=None):
    # Check the image
    if image is None:
        print("Error: No image.")
        return

    # Convert the image to a numpy array if it isn't already
    if not isinstance(image, np.ndarray):
        img = np.array(image)
    else:
        img = image

    # Preparing Canvas
    height, width, _ = img.shape
    longer_side = max(height, width)
    square_canvas = 255 * np.ones((longer_side, longer_side, 3), dtype=np.uint8)

    y_offset = (longer_side - height) // 2
    x_offset = (longer_side - width) // 2

    square_canvas[y_offset:y_offset + height, x_offset:x_offset + width] = img

    # Resize the image to 512x512 pixels
    resized_img = cv2.resize(square_canvas, (512, 512), interpolation=cv2.INTER_AREA)

    return resized_img


In [3]:
pipe1 = None
def init_Sketch2Paint():

    global pipe1
    controlnet = ControlNetModel.from_pretrained(
        "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
    )

    pipe1 = StableDiffusionControlNetPipeline.from_pretrained(
        "Linaqruf/anything-v3.0", controlnet=controlnet, safety_checker=None, torch_dtype=torch.float16
    )

    pipe1.scheduler = UniPCMultistepScheduler.from_config(pipe1.scheduler.config)

    # Remove if you do not have xformers installed
    # see https://huggingface.co/docs/diffusers/v0.13.0/en/optimization/xformers#installing-xformers
    # for installation instructions
    # pipe.enable_xformers_memory_efficient_attention()

    pipe1.enable_model_cpu_offload()


In [7]:
def sketch2paint(img,text,num=20):
    image_re = re_image(image=img)
    image_re0 =Image.fromarray(image_re)

    image = np.array(image_re0)
    low_threshold = 150
    high_threshold = 200

    image = cv2.Canny(image, low_threshold, high_threshold)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    image = Image.fromarray(image)
    print("starting sketch-to-paint process...")
    global pipe1
    image1 = pipe1(text, image, num_inference_steps=int(num)).images[0] #generator=torch.manual_seed(seed)
    image1.save("Colorization_output.png")
    
    return image1

starting sketch-to-paint process...


Traceback (most recent call last):
  File "C:\Users\wz148\AppData\Local\anaconda3\lib\site-packages\gradio\routes.py", line 422, in run_predict
    output = await app.get_blocks().process_api(
  File "C:\Users\wz148\AppData\Local\anaconda3\lib\site-packages\gradio\blocks.py", line 1323, in process_api
    result = await self.call_function(
  File "C:\Users\wz148\AppData\Local\anaconda3\lib\site-packages\gradio\blocks.py", line 1051, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "C:\Users\wz148\AppData\Local\anaconda3\lib\site-packages\anyio\to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "C:\Users\wz148\AppData\Local\anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
  File "C:\Users\wz148\AppData\Local\anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 754, in run
    result = context.run

starting sketch-to-paint process...


Traceback (most recent call last):
  File "C:\Users\wz148\AppData\Local\anaconda3\lib\site-packages\gradio\routes.py", line 422, in run_predict
    output = await app.get_blocks().process_api(
  File "C:\Users\wz148\AppData\Local\anaconda3\lib\site-packages\gradio\blocks.py", line 1323, in process_api
    result = await self.call_function(
  File "C:\Users\wz148\AppData\Local\anaconda3\lib\site-packages\gradio\blocks.py", line 1051, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "C:\Users\wz148\AppData\Local\anaconda3\lib\site-packages\anyio\to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "C:\Users\wz148\AppData\Local\anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
  File "C:\Users\wz148\AppData\Local\anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 754, in run
    result = context.run

## Use it With GUI! 😺

### Start Pipeline

In [5]:
init_Sketch2Paint()

vae\diffusion_pytorch_model.safetensors not found


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

### Start GUI

In [8]:
import numpy as np
import gradio as gr

# def generate(img): #fake generate
#     return img
css='''
.container {max-width: 1150px;margin: auto;padding-top: 1.5rem}
.image_upload{min-height:500px}
.image_upload [data-testid="image"], .image_upload [data-testid="image"] > div{min-height: 500px}
.image_upload [data-testid="sketch"], .image_upload [data-testid="sketch"] > div{min-height: 500px}
.image_upload .touch-none{display: flex}
#output_image{min-height:500px;max-height=500px;}
'''

example_button=gr.Button(label='example',value='Use Result as New Input').style(full_width=False, size='sm')
def example_fill1():

  return Image.open('Sketch_1.jpeg')



with gr.Blocks(theme=gr.themes.Default(font=[gr.themes.GoogleFont("IBM Plex Mono"), "ui-monospace","monospace"],
                                       primary_hue="lime",
                                       secondary_hue="emerald",
                                       neutral_hue="slate",
                                       ), css=css) as demo:
    with gr.Accordion('小提示✨', open=False):
        gr.Markdown('1. 请根据描述自行拆分和添加关键词')
        gr.Markdown('2. 关键词描述前先加入best quality, masterpiece等词以保证生成质量')
        gr.Markdown('3. 关键词添加顺序：先描述人（或者主体），后描述物')
        gr.Markdown('4. 需要英文翻译时可以询问采访者~')
        gr.Markdown('5. 右键单击图像保存到本地')
    with gr.Row().style(equal_height=True):
        with gr.Column():
            gr.Markdown('### 输入线稿和文字')
            example_button=gr.Button(value="用刚刚改好的线稿吧！")
            input_image = gr.Image(shape=(200, 200))
            input_text = gr.Textbox(label="上色文字描述",lines=1)
            generate_button = gr.Button(label="上色🎇", value = "上色🎇")
            

        with gr.Column():  
            gr.Markdown('### 上色结果')   
            output_image = gr.Image() 
            
    

    generate_button.click(fn=sketch2paint, inputs =[input_image, input_text], outputs=[output_image])
    example_button.click(fn=example_fill1, outputs=[input_image])        
# demo = gr.Interface(sketch2paint, [gr.Image(shape=(200, 200)),gr.Textbox(lines=1)], "image",css=css)

demo.queue().launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


starting sketch-to-paint process...


  0%|          | 0/20 [00:00<?, ?it/s]

starting sketch-to-paint process...


  0%|          | 0/20 [00:00<?, ?it/s]

starting sketch-to-paint process...


  0%|          | 0/20 [00:00<?, ?it/s]

### New Feature
#### Resize the sketch input freely
- with canva size fixed (512*512)
   - can be flexible in future
- resize with gradio UI easily
- give more space to background image generation

## ControlNet Inpaint Demo (Experimental)

In [3]:
# !pip install mediapipe

In [1]:
# this code is largely inspired by https://huggingface.co/spaces/hysts/ControlNet-with-Anything-v4/blob/main/app_scribble_interactive.py
# Thank you, hysts!

# import sys
# sys.path.append('./src/ControlNetInpaint/')
# functionality based on https://github.com/mikonvergence/ControlNetInpaint

import gradio as gr
#import torch
#from torch import autocast // only for GPU

from PIL import Image
import numpy as np
from io import BytesIO
import os

# Usage
# 1. Upload image or fill with white
# 2. Sketch the mask (image->[image,mask]
# 3. Sketch the content of the mask

## SETUP PIPE

from diffusers import StableDiffusionInpaintPipeline, ControlNetModel, UniPCMultistepScheduler
from pipeline_stable_diffusion_controlnet_inpaint import *
from diffusers.utils import load_image
from controlnet_aux import HEDdetector

hed = HEDdetector.from_pretrained('lllyasviel/ControlNet')

controlnet = ControlNetModel.from_pretrained(
    "fusing/stable-diffusion-v1-5-controlnet-scribble", torch_dtype=torch.float16
)
pipe = StableDiffusionControlNetInpaintPipeline.from_pretrained(
     "runwayml/stable-diffusion-inpainting", controlnet=controlnet, torch_dtype=torch.float16
 )

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.safety_checker = lambda images, clip_input: (images, False)

if torch.cuda.is_available():
    # Remove if you do not have xformers installed
    # see https://huggingface.co/docs/diffusers/v0.13.0/en/optimization/xformers#installing-xformers
    # for installation instructions
#     pipe.enable_xformers_memory_efficient_attention()

    pipe.to('cuda')



safety_checker\model.safetensors not found


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [2]:
# Functions

css='''
.container {max-width: 1150px;margin: auto;padding-top: 1.5rem}
.image_upload{min-height:500px}
.image_upload [data-testid="image"], .image_upload [data-testid="image"] > div{min-height: 500px}
.image_upload [data-testid="sketch"], .image_upload [data-testid="sketch"] > div{min-height: 500px}
.image_upload .touch-none{display: flex}
#output_image{min-height:500px;max-height=500px;}
'''
 
def get_guide(image):  
    return hed(image,scribble=True)

def create_demo():
    # Global Storage
    CURRENT_IMAGE={'image': None,
                   'mask': None,
                   'guide': None
                }    
    HEIGHT, WIDTH=512,512
    
    RESULT_IMAGE=None
    
    with gr.Blocks(theme=gr.themes.Default(font=[gr.themes.GoogleFont("IBM Plex Mono"), "ui-monospace","monospace"],
                                           primary_hue="lime",
                                           secondary_hue="emerald",
                                           neutral_hue="slate",
                                           ),css=css) as demo:

        gr.Markdown('# 线稿修改✏️')
#         with gr.Accordion('Useful Buttons', open=False):
        with gr.Accordion('小提示✨', open=False):
            gr.Markdown('1. 请根据描述自行拆分和添加关键词')
            gr.Markdown('2. 关键词描述前先加入sketch，lineart等词保证生成结果为线稿，加入best quality, masterpiece等词以保证生成质量')
            gr.Markdown('3. 关键词添加顺序：先描述人（或者主体），后描述物')
            gr.Markdown('4. 需要英文翻译时可以询问采访者~')
            gr.Markdown('5. 右键单击图像保存到本地')
            gr.Markdown('PS：线稿修改结果可能含有部分色彩，但不会影响之后的上色过程')
            gr.Markdown('PPS：进行重新修改时请刷新网页~')
        with gr.Box():
#             gr.Markdown('## Cut ✂️')
#             gr.Markdown('1. Upload your image below')
#             gr.Markdown('2. **Draw the mask** for the region you want changed (Cut ✂️)')
#             gr.Markdown('3. Click `Set Mask` when it is ready!')
#             gr.Markdown('## Sketch ✏️')
#             gr.Markdown('4. Now, you can **sketch a replacement** object! (Sketch ✏️)')
#             gr.Markdown('5. (You can also provide a **text prompt** if you want)')
#             gr.Markdown('6. 🔮 Click `Generate` when ready! ')             
            example_button=gr.Button(label='example',value='继续用上一次的修改😊').style(full_width=False, size='sm')
#             restart_button = gr.Button(label='restart', value = "Restart Everything").style(full_width=False, size='sm')
            
        with gr.Group():
          with gr.Box():
            with gr.Column():
              with gr.Row() as main_blocks:
                  with gr.Column() as step_1:
                      gr.Markdown('### 蒙版输入')   
                      image = gr.Image(source='upload',
                                            shape=[HEIGHT,WIDTH],
                                            type='pil',#numpy',
                                            elem_classes="image_upload",
                                       label='Mask Draw (Cut!)',
                                            tool='sketch',
                                            brush_radius=60).style(height=500)
                      input_image=image
                      mask_button = gr.Button(label='Set Mask', value='设置蒙版')
                  with gr.Column(visible=False) as step_2:     
                      gr.Markdown('### 草稿输入')         
                      sketch = gr.Image(source='upload',
                                            shape=[HEIGHT,WIDTH],
                                            type='pil',#'numpy',
                                        elem_classes="image_upload",
                                            label='画上新东西！',#Fill Draw (Sketch!)
                                            tool='sketch',
                                            brush_radius=10).style(height=500)
                      sketch_image=sketch
                      run_button = gr.Button(label='Generate', value='生成修改结果🎇', variant="primary") 
                      prompt = gr.Textbox(label='Prompt')    
              
                  with gr.Column() as output_step:  
                      gr.Markdown('### 修改结果')   
                      output_image = gr.Gallery(
                                      label="Generated images",
                                      show_label=False,
                                      elem_id="output_image",
                                  ).style(height=500,containter=True)              
              with gr.Accordion('Advanced options', open=False):
                  num_steps = gr.Slider(label='Steps',
                                      minimum=1,
                                      maximum=100,
                                      value=20,
                                      step=1)
                  text_scale = gr.Slider(label='Text Guidance Scale',
                                            minimum=0.1,
                                            maximum=30.0,
                                            value=7.5,
                                            step=0.1)
                  seed = gr.Slider(label='Seed',
                                  minimum=-1,
                                  maximum=2147483647,
                                  step=1,
                                  randomize=True)  
                  
                  sketch_scale = gr.Slider(label='Sketch Guidance Scale',
                                            minimum=0.0,
                                            maximum=1.0,
                                            value=1.0,
                                            step=0.05)
                    

#         with gr.Accordion('More Info', open=False):
#             gr.Markdown('This demo was created by Mikolaj Czerkawski [@mikonvergence](https://twitter.com/mikonvergence) based on the 🌱 open-source implementation of [ControlNetInpaint](https://github.com/mikonvergence/ControlNetInpaint) (diffusers-friendly!).')
#             gr.Markdown('The tool currently only works with image resolution of 512px.')
#             gr.Markdown('💡 To learn more about diffusion with interactive code, check out my open-source ⏩[DiffusionFastForward](https://github.com/mikonvergence/DiffusionFastForward) course. It contains example code, executable notebooks, videos, notes, and a few use cases for training from scratch!')
        
        inputs = [
            sketch_image,
            prompt,
            num_steps,
            text_scale,
            sketch_scale,
            seed
        ]
        
        # STEP 1: Set Mask
        def set_mask(content):
            if content is None:
              gr.Error("You must upload an image first.")
              return {input_image : None,
                      sketch_image : None,
                      step_1: gr.update(visible=True),
                      step_2: gr.update(visible=False)
                  }
                  
            background=np.array(content["image"].convert("RGB").resize((512, 512))) # note: direct numpy seemed buggy
            mask=np.array(content["mask"].convert("RGB").resize((512, 512)))

            if (mask==0).all():
              gr.Error("You must draw a mask for the cut out first.")
              return {input_image : content['image'],
                      sketch_image : None,
                      step_1: gr.update(visible=True),
                      step_2: gr.update(visible=False)
                  }

            mask=1*(mask>0)
            # save vars
            CURRENT_IMAGE['image']=background
            CURRENT_IMAGE['mask']=mask
            
            guide=get_guide(background)
            CURRENT_IMAGE['guide']=np.array(guide)
            guide=255-np.asarray(guide)  
            
            seg_img = guide*(1-mask) + mask*192
            preview = background * (seg_img==255)
            
            vis_image=(preview/2).astype(seg_img.dtype) + seg_img * (seg_img!=255)
            
            return {input_image : content["image"],
                    sketch_image : vis_image,
                    step_1: gr.update(visible=False),
                    step_2: gr.update(visible=True)
                }
        
        # STEP 2: Generate
        def generate(content,
             prompt,
             num_steps,
             text_scale,
             sketch_scale,
             seed):
            sketch=np.array(content["mask"].convert("RGB").resize((512, 512)))            
            sketch=(255*(sketch>0)).astype(CURRENT_IMAGE['image'].dtype) 
            mask=CURRENT_IMAGE['mask']
            
            CURRENT_IMAGE['guide']=(CURRENT_IMAGE['guide']*(mask==0) + sketch*(mask!=0)).astype(CURRENT_IMAGE['image'].dtype)
            
            mask_img=255*CURRENT_IMAGE['mask'].astype(CURRENT_IMAGE['image'].dtype)
            
            new_image = pipe(
              prompt,
              num_inference_steps=num_steps,
              guidance_scale=text_scale,
              generator=torch.manual_seed(seed),
              image=Image.fromarray(CURRENT_IMAGE['image']),
              control_image=Image.fromarray(CURRENT_IMAGE['guide']),
              controlnet_conditioning_scale=sketch_scale,
              mask_image=Image.fromarray(mask_img)
            ).images#[0]
            
            RESULT_IMAGE = new_image
            
            RESULT_IMAGE[0].save("Sketch_1.jpeg")
            
            return {output_image : new_image,
                step_1: gr.update(visible=True),
                step_2: gr.update(visible=False)
                }

        def example_fill():

          return Image.open('Sketch_1.jpeg')
#             if RESULT_IMAGE != None:
#                 print("not a valid image")
#             return RESULT_IMAGE
        
        example_button.click(fn=example_fill, outputs=[input_image])
        mask_button.click(fn=set_mask, inputs=[input_image], outputs=[input_image, sketch_image, step_1,step_2])     
        run_button.click(fn=generate, inputs=inputs, outputs=[output_image, step_1,step_2])
        return demo

In [3]:
if __name__ == '__main__':
    demo = create_demo()
    demo.queue().launch() 

C:\Users\wz148\AppData\Local\anaconda3\lib\site-packages\gradio\components.py:164: UserWarning: Unknown style parameter: containter
  warnings.warn(f"Unknown style parameter: {key}")


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [4]:
# import gradio as gr

# def file_selected(file_input):
#     print("yes, file_selected is invoked")
#     print(process_button)
#     return gr.update(visible=True)
    
# with gr.Blocks() as demo:
#     with gr.Row():
#         with gr.Column(scale=1):
#             gr.Markdown("### Data")
#             file_input = gr.File(label="Select File")
#             process_button = gr.Button("Process", visible=False)

#         with gr.Column(scale=2, min_width=600):
#             gr.Markdown("### Output")
#             result_display = gr.TextArea(default="", label="Result", lines=10, visible=False)

#     file_input.change(fn=file_selected, inputs=file_input, outputs=process_button)
    
# if __name__ == "__main__":
#     demo.launch()    